# Imports

In [11]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import warnings
import data_func.read_data as read_data
warnings.filterwarnings("ignore", category=FutureWarning, module="catboost")



# Load datasets

In [12]:
dataframes = read_data.get_training_data()
X_frames_train = dataframes[0]
Y_frames_train = dataframes[1]
X_frames_test = read_data.get_test_data()


# Data clean up

In [13]:
# making shure that target values line up with x_values

def data_allign(x_train, y_train):

  y_train.dropna(inplace=True)
  combined_data = pd.merge(x_train, y_train, left_on='date_forecast', right_on='time')
  y_train = combined_data['pv_measurement']

  if 'time' and 'pv_measurement' in combined_data.columns:
    combined_data.drop(columns=['time', 'pv_measurement'], inplace=True)
    
  return combined_data, y_train

import data_func.aggregation as data_agg

for i in range(len(X_frames_train)):
    X_frames_train[i] = data_agg.gen_agg(X_frames_train[i], 'mean')
    X_frames_train[i], Y_frames_train[i] = data_allign(X_frames_train[i], Y_frames_train[i])


for j in range(len(X_frames_test)):
    X_frames_test[j] = data_agg.gen_agg(X_frames_test[j], 'mean')

print(len(X_frames_train[0]))
print(len(Y_frames_train[0]))
print(len(X_frames_test[0]))


29667
29667
720


# Feature engineering

In [14]:
import data_func.timeseasonality as DTS
import data_func.date_forecast as DTF
import data_func.one_hot_encoding as OHE

categorical_features = ['dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx', 'precip_type_5min:idx']

for i in range(len(X_frames_train)):
    X_frames_train[i] = DTS.append_seasonal_columns(X_frames_train[i])
    X_frames_train[i] = DTF.date_forecast_columns(X_frames_train[i])
    X_frames_train[i] = OHE.one_hot_encode(X_frames_train[i], categorical_features)
    # X_frames_train[i].drop(columns=['absolute_humidity_2m:gm3'], inplace=True)
    # X_frames_train[i].drop(columns=['air_density_2m:kgm3'], inplace=True)
    # X_frames_train[i]['ceiling_height_agl:m'] = X_frames_train[i]['ceiling_height_agl:m'].fillna(0)
    # X_frames_train[i]['cloud_base_agl:m'] = X_frames_train[i]['cloud_base_agl:m'].fillna(100000)



for i in range(len(X_frames_test)):
    X_frames_test[i] = DTS.append_seasonal_columns(X_frames_test[i])
    X_frames_test[i] = DTF.date_forecast_columns(X_frames_test[i])
    X_frames_test[i] = OHE.one_hot_encode(X_frames_test[i], categorical_features)




# Hyperparameter optimization

In [15]:
import optuna
import catboost as cat # Change to model to optimize

# Split the data into training and validation sets
x_train_a, x_val_a, y_train_a, y_val_a = train_test_split(X_frames_train[0], Y_frames_train[0], test_size=0.17, random_state=None)
x_train_b, x_val_b, y_train_b, y_val_b = train_test_split(X_frames_train[1], Y_frames_train[1], test_size=0.17, random_state=None)
x_train_c, x_val_c, y_train_c, y_val_c = train_test_split(X_frames_train[2], Y_frames_train[2], test_size=0.17, random_state=None)


def objective(trial):
    
  params = {
      "objective": trial.suggest_categorical("objective", ["RMSE", "MAE"]),
      "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
      "depth": trial.suggest_int("depth", 1, 12),
      "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
      "bootstrap_type": trial.suggest_categorical(
          "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
      ),
      "used_ram_limit": "3gb",
  }

  if params["bootstrap_type"] == "Bayesian":
      params["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
  elif params["bootstrap_type"] == "Bernoulli":
      params["subsample"] = trial.suggest_float("subsample", 0.1, 1)


  model_a = cat.CatBoostRegressor(**params) # Change to model to optimize
  model_a.fit(x_train_a, y_train_a)

  # Make predictions on the validation set
  y_pred = model_a.predict(x_val_a)

  # Calculate the Mean Squared Error (MSE) as the metric to optimize
  mse = mean_squared_error(y_val_a, y_pred)

  return mse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

best = study.best_params
# Print the best hyperparameters found
print("Best hyperparameters:", best)

[I 2023-10-27 11:11:22,932] A new study created in memory with name: no-name-e05d2ff3-285b-43c0-a791-5fd7536ab492


Learning rate set to 0.067922
0:	learn: 1170.1908828	total: 6.85ms	remaining: 6.85s
1:	learn: 1129.8060019	total: 22.5ms	remaining: 11.2s
2:	learn: 1087.4119521	total: 39.8ms	remaining: 13.2s
3:	learn: 1053.6121429	total: 55.5ms	remaining: 13.8s
4:	learn: 1023.1027081	total: 71.2ms	remaining: 14.2s
5:	learn: 987.8987837	total: 86.6ms	remaining: 14.3s
6:	learn: 949.5640763	total: 101ms	remaining: 14.3s
7:	learn: 919.1332255	total: 116ms	remaining: 14.4s
8:	learn: 898.4666558	total: 131ms	remaining: 14.5s
9:	learn: 872.0489931	total: 148ms	remaining: 14.6s
10:	learn: 847.2826879	total: 163ms	remaining: 14.6s
11:	learn: 818.9561248	total: 178ms	remaining: 14.6s
12:	learn: 801.1864376	total: 183ms	remaining: 13.9s
13:	learn: 797.9235944	total: 194ms	remaining: 13.6s
14:	learn: 779.2783593	total: 207ms	remaining: 13.6s
15:	learn: 765.8084926	total: 222ms	remaining: 13.7s
16:	learn: 750.4365729	total: 238ms	remaining: 13.8s
17:	learn: 733.4570524	total: 254ms	remaining: 13.8s
18:	learn: 715.

[I 2023-10-27 11:11:35,910] Trial 0 finished with value: 200642.36784850113 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.06619207765494703, 'depth': 3, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.12676809649857243}. Best is trial 0 with value: 200642.36784850113.


999:	learn: 456.4974728	total: 12.7s	remaining: 0us
Learning rate set to 0.067922
0:	learn: 1135.4027556	total: 28.5ms	remaining: 28.5s
1:	learn: 1082.4458568	total: 59.6ms	remaining: 29.8s
2:	learn: 1036.6077786	total: 97.1ms	remaining: 32.3s
3:	learn: 990.0594013	total: 131ms	remaining: 32.6s
4:	learn: 945.7932856	total: 160ms	remaining: 31.9s
5:	learn: 907.4080207	total: 191ms	remaining: 31.7s
6:	learn: 868.9587646	total: 222ms	remaining: 31.5s
7:	learn: 833.0326880	total: 255ms	remaining: 31.6s
8:	learn: 801.4611858	total: 288ms	remaining: 31.7s
9:	learn: 772.5070402	total: 320ms	remaining: 31.7s
10:	learn: 746.4469629	total: 350ms	remaining: 31.5s
11:	learn: 721.9276492	total: 381ms	remaining: 31.4s
12:	learn: 700.2086324	total: 412ms	remaining: 31.3s
13:	learn: 678.6859198	total: 445ms	remaining: 31.3s
14:	learn: 657.5095100	total: 471ms	remaining: 30.9s
15:	learn: 638.6962169	total: 502ms	remaining: 30.9s
16:	learn: 622.8622113	total: 534ms	remaining: 30.9s
17:	learn: 607.860248

[I 2023-10-27 11:12:08,764] Trial 1 finished with value: 133467.7774293086 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.08290847891191566, 'depth': 11, 'boosting_type': 'Plain', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 4.317566404709167}. Best is trial 1 with value: 133467.7774293086.


...

989:	learn: 250.2663082	total: 4.45s	remaining: 45ms
990:	learn: 250.1986533	total: 4.46s	remaining: 40.5ms
991:	learn: 250.1982689	total: 4.46s	remaining: 36ms
992:	learn: 250.1906428	total: 4.47s	remaining: 31.5ms
993:	learn: 250.1894787	total: 4.47s	remaining: 27ms
994:	learn: 250.1858814	total: 4.48s	remaining: 22.5ms
995:	learn: 250.1853655	total: 4.48s	remaining: 18ms
996:	learn: 250.1831997	total: 4.48s	remaining: 13.5ms
997:	learn: 250.1649933	total: 4.5s	remaining: 9.02ms
998:	learn: 249.9526899	total: 4.5s	remaining: 4.51ms
999:	learn: 249.9460823	total: 4.51s	remaining: 0us
Learning rate set to 0.067922
0:	learn: 1142.3975313	total: 3.99ms	remaining: 3.99s
1:	learn: 1095.0763116	total: 20.1ms	remaining: 10s
2:	learn: 1053.2193356	total: 33.3ms	remaining: 11.1s
3:	learn: 1012.4163591	total: 48.4ms	remaining: 12.1s
4:	learn: 1004.3066008	total: 51.8ms	remaining: 10.3s
5:	learn: 1000.1833644	total: 54.8ms	remaining: 9.07s
6:	learn: 964.3577114	total: 67.8ms	remaining: 9.61s
7:	

[I 2023-10-27 11:15:35,414] Trial 8 finished with value: 187660.47262020133 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.024766864845721243, 'depth': 7, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.4981773647226166}. Best is trial 4 with value: 129145.95761257509.


985:	learn: 429.2580424	total: 8.48s	remaining: 120ms
986:	learn: 429.2356867	total: 8.49s	remaining: 112ms
987:	learn: 429.2354566	total: 8.49s	remaining: 103ms
988:	learn: 429.2354566	total: 8.49s	remaining: 94.5ms
989:	learn: 429.2303545	total: 8.51s	remaining: 85.9ms
990:	learn: 429.1441987	total: 8.52s	remaining: 77.4ms
991:	learn: 429.0159362	total: 8.54s	remaining: 68.8ms
992:	learn: 428.9785964	total: 8.55s	remaining: 60.2ms
993:	learn: 428.9555828	total: 8.55s	remaining: 51.6ms
994:	learn: 428.9302250	total: 8.56s	remaining: 43ms
995:	learn: 428.9302249	total: 8.56s	remaining: 34.4ms
996:	learn: 428.9024331	total: 8.57s	remaining: 25.8ms
997:	learn: 428.8402772	total: 8.58s	remaining: 17.2ms
998:	learn: 428.8401253	total: 8.59s	remaining: 8.6ms
999:	learn: 428.7705554	total: 8.6s	remaining: 0us
Learning rate set to 0.067922
0:	learn: 1193.1878037	total: 2.55ms	remaining: 2.54s
1:	learn: 1193.1878037	total: 4.7ms	remaining: 2.34s
2:	learn: 1165.5214339	total: 8.09ms	remaining: 

[I 2023-10-27 11:15:44,628] Trial 9 finished with value: 198251.60994744778 and parameters: {'objective': 'RMSE', 'colsample_bylevel': 0.01670175699390116, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.12386735622688248}. Best is trial 4 with value: 129145.95761257509.


981:	learn: 447.4525567	total: 8.88s	remaining: 163ms
982:	learn: 447.4525567	total: 8.89s	remaining: 154ms
983:	learn: 447.4510629	total: 8.89s	remaining: 145ms
984:	learn: 447.4079406	total: 8.89s	remaining: 135ms
985:	learn: 447.3895361	total: 8.91s	remaining: 126ms
986:	learn: 447.3895148	total: 8.91s	remaining: 117ms
987:	learn: 447.3837331	total: 8.91s	remaining: 108ms
988:	learn: 447.3836784	total: 8.91s	remaining: 99.2ms
989:	learn: 447.3315561	total: 8.93s	remaining: 90.2ms
990:	learn: 447.3315561	total: 8.93s	remaining: 81.1ms
991:	learn: 447.3206112	total: 8.93s	remaining: 72ms
992:	learn: 447.2846274	total: 8.94s	remaining: 63ms
993:	learn: 447.2842018	total: 8.95s	remaining: 54ms
994:	learn: 447.2842018	total: 8.95s	remaining: 45ms
995:	learn: 447.2823733	total: 8.95s	remaining: 36ms
996:	learn: 447.2739006	total: 8.96s	remaining: 27ms
997:	learn: 447.2734168	total: 8.96s	remaining: 18ms
998:	learn: 447.2720439	total: 8.96s	remaining: 8.97ms
999:	learn: 447.2720439	total: 